# <font color='darkorchid'>Wilson Dam</font>
<font color = 'slategray' > This notebook provides analyzes simulates the operations at Wilson dam, using the Canteen package. </font>

by: John Kucharski | 12 July 2021


In [1]:
import numpy as np
import matplotlib.pyplot as plt

from scipy import interpolate

# <font color = darkviolet> Wilson Reservoir </font>
<font color = slategray> This notebook analyzes the Wilson Reservoir in the xx watershed in Kansas. 

In [2]:
import sys
import typing
import datetime as datetime

import numpy as np
import pandas as pd
from scipy import interpolate

In [3]:
sys.path.insert(0, '/Users/johnkucharski/Documents/source/canteen')
import src.data as data
import src.outlet as outlet
import src.reservoir as reservoir
import src.utilities as utilities
import src.operations as operations
import src.simulation as simulation

## <font color = salmon> Global Variables </font>
<font color = slategray> A few global variables used in this notebook are defined here. </font>

In [4]:
data_folder: str = '/Users/johnkucharski/Documents/data/usace/ks/'

## <font color = salmon> Reservoir </font>

### <font color = slateblue> map: volume-elevation (rating curve) </font>
<font color = slategray> A lot of the Wilson dam and reservoir data is expressed in terms of elevation, instead of volume. As a result, the rating curve needs to be established first. Later on, I plot this data. </font> 

In [5]:
rating_path: str = data_folder + 'wilson_rating_curve.csv'

volumes = np.loadtxt(rating_path, skiprows=1, delimiter=',', usecols=[1])      #af
elevations = np.loadtxt(rating_path, skiprows=1, delimiter=',', usecols=[0])   #ft

f_rating = interpolate.interp1d(volumes, elevations, fill_value=(elevations[0], elevations[-1]), bounds_error=False)
inversef_rating = interpolate.interp1d(elevations, volumes, fill_value=(volumes[0], volumes[-1]), bounds_error=False)

rating_map = reservoir.Map(name='elevation', f=f_rating, inverse_f=inversef_rating)

### <font color = slateblue> outlets </font>
<font color = slategray> The reservoir contains two outlets:
 - gate: a single release gate
 - spillway: a spillway with no operational control

In [6]:
gate_path = data_folder + 'wilson_outlets.csv'

elevations = np.loadtxt(gate_path, skiprows=1, delimiter=',', usecols=[0])   # ft
gate_max_cfs = np.loadtxt(gate_path, skiprows=1, delimiter=',', usecols=[1]) # cfs
spillway_cfs = np.loadtxt(gate_path, skiprows=1, delimiter=',', usecols=[2]) # cfs
emergency_cfs = np.loadtxt(gate_path, skiprows=1, delimiter=',', usecols=[3]) # cfs

gate_location, spillway_location, emergency_location = 1450.5, 1545.5, 1582.5
volumes = [inversef_rating(x) for x in elevations]
gate_max_af = [x * utilities.days_to_sec(1) * utilities.cf_to_af(1) for x in gate_max_cfs]
spillway_af = [x * utilities.days_to_sec(1) * utilities.cf_to_af(1) for x in spillway_cfs]
emergency_af = [x * utilities.days_to_sec(1) * utilities.cf_to_af(1) for x in emergency_cfs]

gate = outlet.Outlet('gate', gate_location, utilities.f_interpolate_from_data(volumes, gate_max_af, extrapolate_lo=0, extrapolate_hi=gate_max_af[-1]))
spillway = outlet.Outlet('spillway', spillway_location, utilities.f_interpolate_from_data(volumes, spillway_af, extrapolate_lo=0, extrapolate_hi=spillway_af[-1])) 
emergency = outlet.Outlet('emergency_spillway', emergency_location, utilities.f_interpolate_from_data(volumes, emergency_af, extrapolate_lo=0)) #Note: this will return nan above data range.

wilson_outlets = [gate, spillway, emergency]

<font color = slategray> The following block of code constructs a Canteen _reservoir_ object, for Wilson Dam.

In [7]:
k = float(inversef_rating(1592))
wilson = reservoir.Reservoir(capacity=k, outlets=wilson_outlets, maps={rating_map})
wilson.print()

'default(capacity: 1821585.0, outlets: [emergency_spillway(location: 1582.0), spillway(location: 1546.0), gate(location: 1450.0)], mapped variables: [elevation])'

## <font color = salmon> Operations </font>

<font color = 'slategray'>Wilson reservoir operations in this workbook are based on those found in the 2021 Hydrologic Engineering Center Reservoir Simulation (HEC-RESSIM) model of the Kansas River Reservoirs Flood and Sediment Study. To run a simulation in the Canteen package, all the operations are contained in a single **operate()** function. However, for clarity the componentized HEC-RESSIM model set up is followed below.</font>

### <font color = slateblue>Pools</font>

<font color = slategray>In the HEC-RESSIM model, operational rules are defined for the following zones: 
<ol>
<li> inactive (no operations) pool</li>
<li> conservation pool </li>
<li> flood pool, and </li> 
<li> surcharge space </li>
</ol>

In the HEC-RESSIM model releases above the top of the conservation pool/bottom of the flood pool is determined based on the storage in some of the Kansas River watershed system reservoirs. To reproduce that behavior here would require modeling the entire system, this beyond the scope of this assignment (and would not support the goals of this assignment). The operations here assume a constant top of conservation pool elevation of 1516 ft (as indicated in the HEC-RESSIM model).

The _active_zone()_ function in the next block of code, is used during the simulation to determine which zone of the reservoir is active. It returns an enum provided in the _operations_ module. 

In [8]:
top_inactive_elev_ft = 1440
top_conservation_elev_ft = 1516
surcharge_elevation_ft = 1554.5

def active_zone(input: data.Input) -> operations.Zone:
    if input.storage + input.inflow  < rating_map.inverse_f(top_inactive_elev_ft):
        return operations.Zone.INACTIVE
    elif rating_map.inverse_f(top_inactive_elev_ft) < input.storage + input.inflow < rating_map.inverse_f(top_conservation_elev_ft):
        return operations.Zone.CONSERVATION
    elif rating_map.inverse_f(top_conservation_elev_ft) < input.storage + input.inflow < rating_map.inverse_f(surcharge_elevation_ft):
        return operations.Zone.FLOOD
    else:
        return operations.Zone.SURCHARGE

<font color=slategray> Next operations for each of these pools are defined as seperate functions. All of these operations could be defined in one large function, but this componentization of the code more closely matches the way these rule are defined in the HEC-RESSIM model. Each function takes two arguments: 

1. input (_input.Input_): inputs for a single timestep in the simulation. This input object contains at a minimum:

    * _Input.date_: a date for the simulation timestep, 
    * _Input.storage_: the reservoir storage volume leading into the timestep, and
    * _Input.inflow_: the inflow volume into the reservoir in that timestep
    * optional _Input.additional_arguments_ are discussed later in the notebook.
<br/><br/>
2. outlets (List[outlet.Outlet]): a list of outlets from which the releases will be made, typically Reservoir.outlets

Each fuction returns a dictionary (_Dict[str, float]_) with keys matching the _Outlet.name_ attribute an the release volume values.


#### <font color = slateblue> INACTIVE ZONE

<font color=slategray> Within the inactive zone no releases are made.

In [9]:
def inactive_operations(input: data.Input, outlets: typing.List[outlet.Outlet] = wilson_outlets) -> typing.Dict[str, float]:
    return {x.name: 0.0 for x in outlets}
print(inactive_operations(data.Input(datetime.date(2020, 1, 1), 1, 1), wilson_outlets))

{'gate': 0.0, 'spillway': 0.0, 'emergency_spillway': 0.0}


#### <font color = slateblue> CONSERVATION POOL

<font color=slategray> A simple minimum release, defined as step function (based on time of year) is made from the conseration pool.

In [10]:
def conservation_operations(input: data.Input, outlets: typing.List[outlet.Outlet] = wilson_outlets) -> typing.Dict[str, float]:
    yr: int = input.date.year
    convert = utilities.days_to_sec(1) * utilities.cf_to_af(1)
    release = 15.0 * convert if datetime.date(yr, 4, 1) < input.date < datetime.date(yr, 10, 1) else 5.0 * convert    
    return  {x.name: (min(release, x.max_release(input.storage + input.inflow)) if x.name == 'gate' else 0.0) for x in outlets}  
print(conservation_operations(data.Input(datetime.date(2020, 1, 1), rating_map.inverse_f(1518), 1), wilson_outlets))     

{'gate': 9.917355371900825, 'spillway': 0.0, 'emergency_spillway': 0.0}


#### <font color = slateblue> FLOOD POOL

<font color=slategray> In the HEC-RESSSIM model flood operations depend on storage in other Kansas watershed system reservoirs. Functions for these operations are provided at the end of this workbook, but are beyond the scope (would require modeling the entire system) and needs of this study. For this reason, a simple flood pool rule is provided in the next code block.

In [11]:
def flood_operations(input: data.Input, outlets: typing.List[outlet.Outlet] = wilson_outlets):
    release = {}
    storage = input.storage + input.inflow 
    target = storage - rating_map.inverse_f(top_conservation_elev_ft)
    for x in outlets:
        out = min(x.max_release(storage), target)
        storage, target = storage - out, target - out
        release[x.name] = out
    return release
print(flood_operations(data.Input(datetime.date(2020, 1, 1), rating_map.inverse_f(1550), 1), wilson_outlets))

{'gate': 6489.015732300253, 'spillway': 5643.449921240429, 'emergency_spillway': 0.0}


#### <font color = slateblue> SURCHARGE SPACE

<font color=slategray> In this pool the _flood_release()_ rules still apply. No rules are given for releases above 1588 ft (the top of dam is at 1592), operations at 1588 are assumed for the purposes of this model to extend to the top of dam. Above the top of dam outflow will be assumed to equal inflow.

In [12]:
def surcharge_operations(input: data.Input, outlets: typing.List[outlet.Outlet] = wilson_outlets):
    storage = input.storage + input.inflow
    emergency_release = outlet.select_outlet('emergency_spillway', outlets).max_release(storage)
    storage = storage - emergency_release
    spillway_release = outlet.select_outlet('spillway', outlets).max_release(storage)
    storage = storage - spillway_release
    releases = flood_operations(data.Input(input.date, inflow = 0, storage = storage), outlet.deselect_outlets(['spillway', 'emergency_spillway'], outlets))
    releases['spillway'] = spillway_release
    releases['emergency_spillway'] = emergency_release
    return releases
print(surcharge_operations(data.Input(datetime.date(2020, 1, 1), rating_map.inverse_f(1585), 1), wilson_outlets))

{'gate': 0.0, 'spillway': 14538.506034737462, 'emergency_spillway': 9044.067258729357}


### <font color = slateblue>operate() Function</font>

<font color = slategray>As is mentioned above, the canteen package requires an _operate_ function, which matches the signature of the pool release functions defined above. In other words it must take: (1) an input (_input.Input_), and (2) outlets (_List[outlet.Outlet]_) as arguments and return a _Dict[str, float]_ dictionary of _Outlet.name_ keys with release volume value pairs. This can be a single function that contains all the reservoir's operational rules. 

In this case, Wilson dam's operational rules are defined above across multiple functions (namely: _inactive_operations()_, _conservation_operations()_, _flood_operations()_, _surcharge_operations()_). In particular, a single function is defined for each of the reservoir's active zones. The active zones are in turn defined by the _active_zone()_ function. 

The _operations_ module contains an _Operations_ base class that contains the required _operate()_ function. Its constructor, accepts a list of rules, matching the _operate()_ function's signature. A _Wilson_Operations_ subclass that inherits from the _Operations_ base class is createed below, its constructor takes the _active_zone()_ function as an arguement, in addition to the rules required by baseclass. This _active_zone()_ function and operational rules are used to fill out its _operate()_ function.

In [13]:

wilson_rules = [inactive_operations, conservation_operations, flood_operations, surcharge_operations] 
class Wilson_Operations(operations.Operations):
    def __init__(self, rules: typing.List[typing.Callable[[data.Input, typing.List[outlet.Outlet]], typing.Dict[str, float]]] = wilson_rules,
                 wilson_active_zone: typing.Callable[[data.Input], operations.Zone] = active_zone) -> None:
        self.active_zone = wilson_active_zone
        super().__init__(rules)
    
    @property
    def rules(self):
        return super().rules
    
    def operate(self, input: data.Input, outlets: typing.List[outlet.Outlet]) -> typing.Dict[str, float]:
        zone = self.active_zone(input)
        if zone == operations.Zone.INACTIVE:
            return self.rules[0](input, outlets)
        elif zone == operations.Zone.CONSERVATION:
            return self.rules[1](input, outlets)
        elif zone == operations.Zone.FLOOD:
            return self.rules[2](input, outlets)
        else: #zone == operatins.Zone.SURCHARGE
            return self.rules[3](input, outlets)
              
wilson_ops = Wilson_Operations(wilson_rules)
print(wilson_ops.operate(data.Input(datetime.datetime(2021, 9, 3), 100), wilson_outlets))

{'gate': 0.0, 'spillway': 14717.355371900825, 'emergency_spillway': nan}


## <font color = salmon> Simulation </font>

<font color = 'slategray'>A simulation is run using the _simulate()_ function. This function takes the following arguments:
    
1. **wilson**: _Reservoir_ object (described above), 
2. **wilson_ops**: _operate()_ function (also described above), and
3. a list of data **inputs**: _List[input.Input]_ (described below)
</font>


### <font color = slateblue>Input</font>

<font color = slategray>The inputs for a simulation are provided as a list of inputs (e.g. _List[inputs.Input]_). Each _Input_ item in this list serves as the input for a timestep in the simulation model. An _Input_ contains at a minimum:

1. _date_: a _datetime.date_ for the simulation timestep. </li>
2. _inflow_: an inflow volume at the _Reservoir_ during the simulation timestep. </li>
3. _storage_: an storage volume in the reservoir at the beginning of timestep. </li> 
4. _additional_arguments_: an optional _Dict[str, float]_ of input key, value pairs described below. </li>

In this case the _date_ and _inflow_ parameters for the list of _Input_ are taken from the _dam_inflows.csv_ data. The simulation is initialized with the _Input.storage_ value equivalent to the top of conservation pool. Each subsquent _Input.storage_ value is updated within the _simulate()_ function. The np.nan value is used to initialize each _Input.storage_ value. A function for importing the _dam_inflows.csv_ data, and generating the _storage_ values is presented and run below. Then the first 5 _Input_ values are displayed.

In [14]:
def import_wilson_csv(path: str, initial_storage: float) -> pd.DataFrame:
    df = pd.read_csv(path, header=0, names=['_', 'date', 'inflow_cfs', '__'], skiprows=[1, 2, 3], skipinitialspace=True, 
                     usecols=['date', 'inflow_cfs'], dtype={'inflow_cfs': float}, parse_dates=True)
    df.dropna(axis='rows', inplace=True)
    df.reset_index(drop=True, inplace =True)
    df['date'] = pd.to_datetime([x.rsplit(',', maxsplit=1)[0] for x in df['date'].to_list()])
    df['inflow_af'] = [(x * utilities.days_to_sec(1) * utilities.cf_to_af(1)) for x in df.inflow_cfs.to_numpy()]
    df['storage_af'] = np.concatenate(([initial_storage], np.full(df.shape[0] - 1, np.nan)))
    df.drop(columns=['inflow_cfs'], inplace=True)
    return df
wilson_input_data = import_wilson_csv(data_folder + 'dam_inflows.csv', rating_map.inverse_f(top_conservation_elev_ft))
wilson_input_data.head()

,date,inflow_af,storage_af
0,1919-08-30,35.702479,236188.0
1,1919-08-31,35.702479,NaN
2,1919-09-01,35.702479,NaN
3,1919-09-02,35.702479,NaN
4,1919-09-03,43.636364,NaN


In [15]:
wilson_inputs = [data.Input(date=row['date'], inflow=row['inflow_af'], storage=row['storage_af'], update_storage=True) for idx, row in wilson_input_data.iterrows()]
wilson_simulation = simulation.Simulation(wilson_inputs, wilson, wilson_ops.operate)
outs = wilson_simulation.simulate()

{'emergency_spillway': 0.0, 'spillway': 0.0, 'gate': 35.70247933885548}


TypeError: unsupported operand type(s) for -: 'Input' and 'float'

In [ ]:
outflow: typing.Dict[str, typing.List[float]] = {'a': list(), 'b':[1, 2]}
r = {'a': 1, 'c': 3}
outflow.update({ k: outflow[k] + [v] if k in outflow else [v] for k, v in r.items() })
print(outflow)

The 

In [ ]:
yr: int = 2021 #required by datetime module but not used here.
may_to_june_elev_ft, rest_of_yr_elev_ft = 1539, 1531
rule_curve = operations.Rule_Curve([(datetime.date(yr, 1, 1), rest_of_yr_elev_ft), (datetime.date(yr, 4, 1), rest_of_yr_elev_ft),
                                    (datetime.date(yr, 5, 1), may_to_june_elev_ft), (datetime.date(yr, 6, 1), may_to_june_elev_ft),
                                    (datetime.date(yr, 7, 1), rest_of_yr_elev_ft)])